In [41]:
import pandas as pd
import numpy as np
from modules import preprocess


static_features = ["awc", "bulk_density", "drainage_class_1", "drainage_class_2", "drainage_class_3", "drainage_class_4", "drainage_class_5", "drainage_class_6"]
temporal_prefixes = ["tavg", "prec", "tmin", "tmax", "ndvi", "fpar", "rad", "et0", "cwb", "ssm", "rsm"]
# Enable autoreload for Jupyter notebooks
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Define study crop and country

In [2]:
# USER INPUTS
country = "US" # one of ["US", "BR"]
crop = "wheat" # one of ["maize", "wheat"]

shapefile_path, crop_season_in_days_of_year, crop_season_in_months, _ = preprocess.get_study_metadata(country, crop)

print(shapefile_path, crop, country, crop_season_in_days_of_year, crop_season_in_months)

../data/shapefiles/US/tl_2023_us_county/tl_2023_us_county.shp wheat US (9, 233) (2, 8)


### 2. Read data

From CY-Bench, we have five predictor datasets. 

| **ID** | **Name**        | **Time** | **Variables**                         | **Steps**                                    | **Notebook**   |
|--------|-----------------|----------|---------------------------------------|----------------------------------------------|----------------|
| 1      | FPAR            | bins     | fpar                                  | filter adm_ids, filter crop season           | _this notebook_ |            
| 2      | METEO           | daily    | tmin, tmax, prec, rad, tavg, et0, cwb | filter adm_ids, filter crop season,<br>resample to 8-day bins  | _3_ecmwf_preprocessing.ipynb (tmin, tmax, tavg, prec_) <br>_this notebook_ (et0, rad, cwb) |       
| 3      | NDVI            | bins     | ndvi                                  | filter adm_ids, filter crop season, pivotieren           | _this notebook_ |           
| 4      | SOIL MOISTURE   | daily    | ssm, rsm                              | filter adm_ids, filter crop season, <br>resample to 8-day bins, pivotieren  | _this notebook_ |          
| 5      | SOIL            | static   | awc, bulk_density, drainage_class     | filter adm_ids, pivotieren                                | _this notebook_ |         


In [6]:
# CY-Bench data
fpar = pd.read_csv("../data/CY-Bench/{}/{}/fpar_{}_{}.csv".format(country, crop, crop, country)) 
meteo = pd.read_csv("../data/CY-Bench/{}/{}/meteo_{}_{}.csv".format(country, crop, crop, country), usecols=['crop_name','adm_id','date','rad','et0','cwb', 'tavg', 'tmin', 'tmax', 'prec'])
ndvi = pd.read_csv("../data/CY-Bench/{}/{}/ndvi_{}_{}.csv".format(country, crop, crop, country))
soil_moisture = pd.read_csv("../data/CY-Bench/{}/{}/soil_moisture_{}_{}.csv".format(country, crop, crop, country))
soil = pd.read_csv("../data/CY-Bench/{}/{}/soil_{}_{}.csv".format(country, crop, crop, country))
yield_data = pd.read_csv("../data/CY-Bench/{}/{}/yield_{}_{}.csv".format(country, crop, crop, country))

### 3. Preprocess

In [7]:
cy_bench_data = [fpar, soil_moisture, ndvi, soil, meteo]
relevant_adm_ids = yield_data["adm_id"].unique()
cy_bench_data = preprocess.filter_predictors_by_adm_ids(cy_bench_data, relevant_adm_ids)
fpar, soil_moisture, ndvi, soil, meteo = cy_bench_data

In [12]:
crop_season_in_days_of_year = tuple([1, 365])

#### ToDos

- don't filter crop season
- add fpar interpolation
- provide two dataframes (8-day and 16-day bins)

In [15]:
temporal_cy_bench_data = [fpar, soil_moisture, ndvi, meteo]
temporal_cy_bench_data = preprocess.preprocess_temporal_data(temporal_cy_bench_data, crop_season_in_days_of_year)

fpar, soil_moisture, ndvi, meteo = temporal_cy_bench_data

agg
assign
agg
assign
agg
assign
agg
assign


In [77]:
def interpolate_fpar_timesteps(df, reference_quantity):
    """
    interpolate fpar values for all timesteps of a reference quantity.
    
    Params:
        df: pd.DataFrame, dataframe containing the fpar columns
        reference_quantity: str, reference quantity with complete timesteps
        
    Returns:
        df: pd.DataFrame, dataframe with interpolated fpar values
    
    """
    min_time_step = min([int(c.split("_")[1]) for c in [l for l in df.columns if reference_quantity in l]])
    max_time_step = max([int(c.split("_")[1]) for c in [l for l in df.columns if reference_quantity in l]])
    fpar_columns = [c for c in df.columns if "fpar" in c]
    fpar_columns_all_timesteps = ["fpar_{}".format(n) for n in list(range(min_time_step, max_time_step + 1))]
    new_fpar_columns = list(set(fpar_columns_all_timesteps).difference(set(fpar_columns)))
    df[new_fpar_columns] = np.nan
    df[fpar_columns_all_timesteps] = df[fpar_columns_all_timesteps].interpolate(method='linear', axis=1, limit_direction='both')
    df = df.reindex(sorted(df.columns), axis=1)
    
    return df

def temporal_aggregation(df, feature_prefix_list, window_size):
    """
    Aggregate steps of temporal features and return the resulting dataframe.
    
    Params:
        df: pd.DataFrame, dataframe containing the features and time steps as column names
        feature_prefix_list: list, list of prefixes of temporal features that should be aggregated
        window_size: int, number of steps to aggregate by 
    """
    
    df = df.set_index(static_features + ["adm_id", "harvest_year"], append=False)
    li = []
    for feature in feature_prefix_list:
        res = df[get_temporal_feature_subset(df, [feature])].rolling(window=window_size, min_periods=1, axis=1).mean()
        res = res.iloc[:, 1::2]
        li.append(res)
    
    df = pd.concat(li, axis=1).reset_index()
    return df

def get_temporal_feature_subset(df, feature_prefix_list):
    """
    Filter temporal features by prefix and return the resulting list of features names.
    
    Params:
        df: pd.DataFrame, dataframe containing the features as column names
        feature_prefix_list: list, list of prefixes to filter by
    
    Returns:
        filtered_temporal_features: list, list of filtered temporal feature names
    """
    all_columns = df.columns
    filtered_temporal_features = [f for f in all_columns if any([f.startswith(prefix) for prefix in feature_prefix_list])]
    return filtered_temporal_features

def order_temporal_features(df, temporal_prefixes):
    candidate_columns = [c for c in df.columns if c.split("_")[0] in temporal_prefixes]
    formated_candidate_columns = format_column_names(candidate_columns)
    df = df.rename(columns=dict(zip(candidate_columns, formated_candidate_columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    df = df.set_index(["harvest_year"], append=True).reset_index(level=[1])
    return df

def format_column_names(column_names):
    formatted_columns = []
    for name in column_names:
        prefix, number = name.split('_')
        formatted_name = f"{prefix}_{int(number):02d}"
        formatted_columns.append(formatted_name)
    return formatted_columns

In [22]:
# merge
predictors = (soil_moisture
              .merge(ndvi
                     .merge(meteo
                            .merge(fpar, on=["adm_id", "harvest_year"], how="inner"), 
                            on=["adm_id", "harvest_year"], how="inner"), 
                     on=["adm_id", "harvest_year"], how="inner")
              .merge(soil.drop("crop_name", axis=1), on="adm_id", how="left"))

# one-hot encode drainage_class and drop
predictors = (predictors
              .join(pd.get_dummies(predictors["drainage_class"].astype('Int64'), dtype=int, prefix="drainage_class"))
              .drop("drainage_class", axis=1))

In [29]:
predictors = interpolate_fpar_timesteps(predictors, "tavg")

In [35]:
predictors = order_temporal_features(predictors, temporal_prefixes)

In [78]:
predictors_16day_bins = temporal_aggregation(predictors, temporal_prefixes, 2)

In [79]:
predictors_16day_bins.head(1)

,awc,bulk_density,drainage_class_1,drainage_class_2,drainage_class_3,drainage_class_4,drainage_class_5,drainage_class_6,adm_id,harvest_year,...,rsm_27,rsm_29,rsm_31,rsm_33,rsm_35,rsm_37,rsm_39,rsm_41,rsm_43,rsm_45
0,11.436359,1.342325,0,0,0,1,0,0,US-01-001,2004,...,382.726433,378.93025,390.162439,392.782368,371.961714,387.878651,413.388439,431.495993,446.502943,444.545958


In [80]:
predictors.to_csv("../data/preprocessed/{}/cy_bench_8daybins_{}_{}.csv".format(country, crop, country), index=False)
predictors_16day_bins.to_csv("../data/preprocessed/{}/cy_bench_16daybins_{}_{}.csv".format(country, crop, country), index=False)